# Ibovespa prediction / Feature Selection

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
%matplotlib inline

n_jobs = 15 # number of parallel jobs to run in parallel

## Loading Dataset

In [2]:
data = pd.read_csv('dataset2.csv', index_col=0)
data.index = data.index.astype('datetime64[ns]')
date_index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
data = data.reindex(date_index)

In [3]:
data['IBOV_Direction'] = [1 if x >= 0 else 0 for x in data['IBOV_Close'].diff()]

In [4]:
lag_features = [
    'IBOV_Direction',
    'IBOV_Open',
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close',
    'IBOV_Volume'
]

wma_features = [
    'IBOV_Open',
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close'
]

for feature in lag_features:
    for i in range(1, 8):
        data[f'{feature}_L{i}'] = data[feature].shift(periods=i)
        
for feature in wma_features:    
    data[f'{feature}_WMA30'] = data[feature].shift(periods=1).rolling(
        window=30,
        center=False
    ).apply(lambda x: np.sum(np.arange(1, 31) * x) / np.sum(np.arange(1, 31)), raw=False)
    
data = data.iloc[30:,:]
data.drop([
    'IBOV_Volume',
    'Crude_Oil_Close',
    'Gold_Close',
    'Nasdaq_Close',
    'Dow_Jones_Close',
    'S&P500_Close'
], axis=1, inplace=True)

## Checking for empty values

In [5]:
(data.isnull().sum() > 0).sum()

0

There are no empty values in the dataset

# Feature Selection

## Genetic Filter

In [6]:
import pygad
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import pearsonr
from sklearn.feature_selection import f_regression

def genetic_filter(X, y):
    def f(i, j):
        mi = mutual_info_regression(i.reshape(-1,1), j)[0]
        F, _ = f_regression(i.reshape(-1, 1), j)
        corr, _ = pearsonr(i, j)

        return mi + F[0] + abs(corr)

    f_values = {}
    ncols = X.shape[1]

    results = Parallel(n_jobs=n_jobs)(delayed(f)(X[:,i], X[:,j]) for i in range(ncols - 1) for j in range(i + 1, ncols))

    index = 0
    for i in range(ncols - 1):
        for j in range(i + 1, ncols):
            f_values[(i, j)] = results[index]
            f_values[(j, i)] = results[index]
            index += 1

    results = Parallel(n_jobs=n_jobs)(delayed(f)(y, X[:,i]) for i in range(ncols))
    index = 0
    for i in range(ncols):
        f_values[('target', i)] = results[index]
        index += 1
        
    def fitness_func(solution, solution_idx):
        idx_selected = np.nonzero(solution)[0]

        f_features_target = 0
        for idx in idx_selected:
            f_features_target += f_values[('target', idx)]

        f_features = 0
        for i in range(len(idx_selected) - 1):
            for j in range(i + 1, len(idx_selected)):
                f_features += f_values[(idx_selected[i], idx_selected[j])]

        return f_features_target - f_features

    ga = pygad.GA(
        num_parents_mating=4,
        keep_parents=3,
        sol_per_pop=100,
        num_generations=500,
        num_genes=ncols,
        crossover_type='two_points',
        mutation_type='random',
        mutation_probability=0.001,
        parent_selection_type='rws',
        gene_space=(0, 1),
        fitness_func=fitness_func,
        parallel_processing=['thread', 15]
    )

    ga.run()
    
    return ga

# Model training/validation

## Train/Test split function definition

In [7]:
def train_test_split(X, y, train_size=0.8):
    nrows = X.shape[0]
    sep = math.ceil(nrows * train_size)
    
    return X[:sep,:], y[:sep], X[sep:,:], y[sep:]

## SVM

In [8]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, accuracy_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler

target = 'IBOV_Direction'
X = data.drop([
    target,
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close'
], axis=1).values
y = data[target].values
X_train, y_train, X_test, y_test = train_test_split(X, y)

def run_svm(X_train_, y_train_, X_test_, y_test_):
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('SVM', SVC(
            kernel='poly',
            degree=1
        ))
    ])

    pipe.fit(X_train_, y_train_)
    y_pred = pipe.predict(X_test_)
    
    fpr, tpr, _ = roc_curve(y_test_, y_pred)
    return auc(fpr, tpr), accuracy_score(y_test_, y_pred)

results = Parallel(n_jobs=n_jobs)(delayed(run_svm)(X_train, y_train, X_test, y_test) for i in range(50))
aucs, accs = zip(*results)
print(f'SVM AUC: {np.mean(aucs)} +- {np.std(aucs)}')
print(f'SVM Accuracy: {np.mean(accs)} +- {np.std(accs)}')

SVM AUC: 0.6428329173357074 +- 1.1102230246251565e-16
SVM Accuracy: 0.6434262948207171 +- 0.0


In [9]:
ga = genetic_filter(X, y)

In [10]:
filter_selected = np.nonzero(ga.best_solution()[0])[0]

In [11]:
X_train_fs = X_train[:,filter_selected]
X_test_fs = X_test[:,filter_selected]

results = Parallel(n_jobs=n_jobs)(delayed(run_svm)(X_train_fs, y_train, X_test_fs, y_test) for i in range(50))
aucs, accs = zip(*results)
print(f'SVM AUC: {np.mean(aucs)} +- {np.std(aucs)}')
print(f'SVM Accuracy: {np.mean(accs)} +- {np.std(accs)}')

SVM AUC: 0.6428329173357074 +- 1.1102230246251565e-16
SVM Accuracy: 0.6434262948207171 +- 0.0


## ANN - MLP

In [12]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc
from itertools import combinations_with_replacement
from joblib import Parallel, delayed

target = 'IBOV_Direction'
X = data.drop([
    target,
    'IBOV_High',
    'IBOV_Low',
    'IBOV_Close'
], axis=1).values
y = data[target].values
X_train, y_train, X_test, y_test = train_test_split(X, y)

n_layers = np.arange(2) + 1
n_neurons = np.arange(0, 35, 5) + 5
epochs = [500]
combinations = []

for layers in n_layers:
    combinations.extend(combinations_with_replacement(n_neurons, int(layers)))
    
def run_mlp(X_train, y_train, X_test, y_test, layer_sizes, epochs):
    aucs = []
    accs = []
    
    for i in range(50):
        pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('MLP', MLPClassifier(
                hidden_layer_sizes=layer_sizes,
                solver='adam',
                max_iter=epochs,
                activation='tanh'
            ))
        ])

        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        aucs.append(auc(fpr, tpr))
        accs.append(accuracy_score(y_test, y_pred))
    
    return epochs, layer_sizes, aucs, accs
    
results = Parallel(n_jobs=n_jobs)(delayed(run_mlp)(
    X_train,
    y_train,
    X_test,
    y_test,
    ls,
    ep
) for ep in epochs for ls in combinations)

In [13]:
data = []
for result in results:
    data.append([
        result[0],
        result[1],
        f'{np.mean(result[2])} +- {np.std(result[2])}',
        f'{np.mean(result[3])} +- {np.std(result[3])}'
    ])
    
summary = pd.DataFrame(data, columns=['Epochs', 'Hidden Layer Sizes', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Hidden Layer Sizes,AUC,Accuracy
0,500,"(5,)",0.6357412971706863 +- 0.009076715731594917,0.6375697211155378 +- 0.007737578789149079
9,500,"(5, 15)",0.6344998224723508 +- 0.00718060057710172,0.6366799468791501 +- 0.006519975737239594
12,500,"(5, 30)",0.6340741270674023 +- 0.00822719466707646,0.6365073041168658 +- 0.0069971598367431255
7,500,"(5, 5)",0.6338737063003768 +- 0.00833549900810376,0.6362151394422311 +- 0.008190791082129901
10,500,"(5, 20)",0.6335326412404678 +- 0.008920431754325076,0.6357370517928286 +- 0.007794805982066207
11,500,"(5, 25)",0.6328526838471037 +- 0.010737745205380938,0.6353253652058433 +- 0.009007078330843642
8,500,"(5, 10)",0.6325533245364409 +- 0.01120313814478427,0.6346746347941566 +- 0.010484380815917299
2,500,"(15,)",0.6331826733014314 +- 0.011298237927558385,0.6343559096945551 +- 0.010606401143528744
1,500,"(10,)",0.6334002639067741 +- 0.009250896840005227,0.6341699867197875 +- 0.008918935731986335
13,500,"(5, 35)",0.6302309979173623 +- 0.011370535516443559,0.6327755644090305 +- 0.009865830436866875


In [14]:
ga = genetic_filter(X, y)

In [15]:
filter_selected = np.nonzero(ga.best_solution()[0])[0]

In [16]:
X_train_fs = X_train[:,filter_selected]
X_test_fs = X_test[:,filter_selected]

results = Parallel(n_jobs=n_jobs)(delayed(run_mlp)(
    X_train_fs,
    y_train,
    X_test_fs,
    y_test,
    ls,
    ep
) for ep in epochs for ls in combinations)

In [17]:
data = []
for result in results:
    data.append([
        result[0],
        result[1],
        f'{np.mean(result[2])} +- {np.std(result[2])}',
        f'{np.mean(result[3])} +- {np.std(result[3])}'
    ])
    
summary = pd.DataFrame(data, columns=['Epochs', 'Hidden Layer Sizes', 'AUC', 'Accuracy'])
summary.sort_values(['Accuracy', 'AUC'], ascending=False)

,Epochs,Hidden Layer Sizes,AUC,Accuracy
9,500,"(5, 15)",0.6428107131312167 +- 0.0006367209330184244,0.6434395750332006 +- 0.0006704863512012922
10,500,"(5, 20)",0.6426906302496515 +- 0.0017653256629810079,0.6433598937583 +- 0.0016547059487884597
12,500,"(5, 30)",0.6419160267722294 +- 0.004338507750536517,0.6426029216467463 +- 0.0038295859319193593
6,500,"(35,)",0.6420077582232397 +- 0.0023603914906992264,0.6424833997343957 +- 0.002721925257565087
7,500,"(5, 5)",0.6416025182429533 +- 0.00456309396039908,0.6423904382470118 +- 0.004041550724963233
13,500,"(5, 35)",0.6414686570960716 +- 0.0044376271456489005,0.6423107569721115 +- 0.0038977819554516066
30,500,"(25, 30)",0.6415606005203945 +- 0.0038381653886723657,0.6419787516600265 +- 0.004337346673626781
17,500,"(10, 25)",0.641212911294468 +- 0.004764104532385342,0.641965471447543 +- 0.004313514414053573
23,500,"(15, 30)",0.6412676534024366 +- 0.004478939282638067,0.6418725099601593 +- 0.004279876859534697
15,500,"(10, 15)",0.6411213918167702 +- 0.004401661481247039,0.6418725099601593 +- 0.00384578536331601


# Ensemble A

In [18]:
#Todo